In [3]:
# Imports
from buildingmotif import BuildingMOTIF
from rdflib import Namespace
from buildingmotif.dataclasses import Model
import csv


# create the namespace
BLDG = Namespace('urn:bldg/')
bm = BuildingMOTIF("sqlite://") # in-memory instance

# Create the model! This will raise an exception if the namespace is not syntactically valid.
model = Model.create(BLDG, description="Test Model") 

# Should add volttron configs to triple information

In [53]:
# funcs
def read_csv(path):
    line = 0
    contents = []
    with open(path, 'r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        for row in csv_reader:
            contents.append(row)
        return contents


def create_relation():
    pass

def instantiate_equips():
    pass

def add_points():
    pass


In [55]:
filename  = '/Users/lazlopaul/Desktop/VolttronSemantics/files/entity_relations.csv'
contents = read_csv(filename)
print(contents)

[{'subjectURI': 'FCU1', 'relation': 'feeds', 'objectURI': 'Zone1'}, {'subjectURI': 'FCU2', 'relation': 'feeds', 'objectURI': 'Zone1'}, {'subjectURI': 'FCU3', 'relation': 'feeds', 'objectURI': 'Zone1'}, {'subjectURI': 'FCU4', 'relation': 'feeds', 'objectURI': 'Zone1'}, {'subjectURI': 'FCU5', 'relation': 'feeds', 'objectURI': 'Zone1'}, {'subjectURI': 'Furnace1', 'relation': 'feeds', 'objectURI': 'Zone1'}, {'subjectURI': 'Furnace2', 'relation': 'feeds', 'objectURI': 'Zone1'}, {'subjectURI': 'Furnace1', 'relation': 'hasPart', 'objectURI': 'Boiler1,Fan1,HeatEx1'}, {'subjectURI': 'Furnace2', 'relation': 'hasPart', 'objectURI': 'Boiler2,Fan2,HeatEx2'}, {'subjectURI': 'Boiler1', 'relation': 'feeds', 'objectURI': 'HeatEx1'}, {'subjectURI': 'HeatEx1', 'relation': 'feeds', 'objectURI': 'Fan1'}, {'subjectURI': 'Boiler2', 'relation': 'feeds', 'objectURI': 'HeatEx2'}, {'subjectURI': 'HeatEx2', 'relation': 'feeds', 'objectURI': 'Fan2'}, {'subjectURI': 'Bldg', 'relation': 'hasPart', 'objectURI': 'Offi

In [ ]:
#Instantiate Every thing in table (URI and Type)
#Then create relations
#Then create points
#Then add the topic names for points to them